# Eclipsing binary: Laplace approximation

In this notebook, we're continuing our tutorial on how to do inference. In the [last notebook](EclipsingBinary_FullSolution.ipynb), we solved for *everything* at once using `pymc3`.

**Note that since we're using pymc3, we need to enable lazy evaluation mode in starry.**

In [ ]:
%matplotlib inline

In [ ]:
%run notebook_setup.py

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pymc3 as pm
import exoplanet as xo
import os
import starry
from corner import corner
import theano.tensor as tt
from tqdm.notebook import tqdm

np.random.seed(12)
starry.config.lazy = True
starry.config.quiet = True

## Load the data

Let's load the EB dataset:

In [ ]:
# Run the Generate notebook if needed
if not os.path.exists("eb.npz"):
    import nbformat
    from nbconvert.preprocessors import ExecutePreprocessor

    with open("EclipsingBinary_Generate.ipynb") as f:
        nb = nbformat.read(f, as_version=4)
    ep = ExecutePreprocessor(timeout=600, kernel_name="python3")
    ep.preprocess(nb);

In [ ]:
data = np.load("eb.npz", allow_pickle=True)
A = data["A"].item()
B = data["B"].item()
t = data["t"]
flux = data["flux"]
sigma = data["sigma"]

In [ ]:
fig, ax = plt.subplots(1, figsize=(12, 5))
ax.plot(t, flux, "k.", alpha=0.5, ms=4)
ax.set_xlabel("time [days]", fontsize=24)
ax.set_ylabel("normalized flux", fontsize=24);

In [ ]:
with pm.Model() as model:

    # Force > 0 for some paramss
    PositiveNormal = pm.Bound(pm.Normal, lower=0.0)

    # Primary
    A_inc = pm.Normal("A_inc", mu=80, sd=5, testval=80)
    A_amp = 1.0
    A_r = PositiveNormal("A_r", mu=0.95, sd=0.1, testval=0.95)
    A_m = PositiveNormal("A_m", mu=1.05, sd=0.1, testval=1.05)
    A_prot = PositiveNormal("A_prot", mu=1.25, sd=0.01, testval=1.25)
    pri = starry.Primary(
        starry.Map(ydeg=A["ydeg"], udeg=A["udeg"], inc=A_inc),
        r=A_r,
        m=A_m,
        prot=A_prot,
    )
    pri.map[1] = A["u"][0]
    pri.map[2] = A["u"][1]

    # Secondary
    B_inc = pm.Normal("B_inc", mu=80, sd=5, testval=80)
    B_amp = 0.1
    B_r = PositiveNormal("B_r", mu=0.75, sd=0.1, testval=0.75)
    B_m = PositiveNormal("B_m", mu=0.70, sd=0.1, testval=0.70)
    B_prot = PositiveNormal("B_prot", mu=0.625, sd=0.01, testval=0.625)
    B_porb = PositiveNormal("B_porb", mu=1.01, sd=0.01, testval=1.01)
    B_t0 = pm.Normal("B_t0", mu=0.15, sd=0.001, testval=0.15)
    sec = starry.Secondary(
        starry.Map(ydeg=B["ydeg"], udeg=B["udeg"], inc=B_inc),
        r=B_r,
        m=B_m,
        porb=B_porb,
        prot=B_prot,
        t0=B_t0,
        inc=B_inc,
    )
    sec.map[1] = B["u"][0]
    sec.map[2] = B["u"][1]

    # System
    sys = starry.System(pri, sec)

Now let's define our likelihood function. Normally we would call something like `pm.Normal` to define our data likelihood, but since we're analytically marginalizing over the surface maps, we use the `lnlike` method in the `System` object instead. We pass this to `pymc3` via a `Potential`, which allows us to define a custom likelihood function. Note that in order for this to work we need to set gaussian priors on the spherical harmonic coefficients; see the [previous notebook](EclipsingBinary_Linear.ipynb) for more details.

Note, also, that at no point have we set the map coefficients or the amplitude of either object. These are never used -- we're marginalizing over them analytically!

In [ ]:
with model:
    sys.set_data(flux, C=sigma ** 2)

    # Prior on primary
    pri_mu = np.zeros(pri.map.Ny)
    pri_mu[0] = 1.0
    pri_L = np.zeros(pri.map.Ny)
    pri_L[0] = 1e-2
    pri_L[1:] = 1e-2
    pri.map.set_prior(mu=pri_mu, L=pri_L)

    # Prior on secondary
    sec_mu = np.zeros(pri.map.Ny)
    sec_mu[0] = 0.1
    sec_L = np.zeros(pri.map.Ny)
    sec_L[0] = 1e-4
    sec_L[1:] = 1e-4
    sec.map.set_prior(mu=sec_mu, L=sec_L)

    lnlike = pm.Potential("marginal", sys.lnlike(t=t))

Now that we've specified the model, it's a good idea to run a quick gradient descent to find the MAP (maximum a posteriori) solution. This will give us a decent starting point for the inference problem.

In [ ]:
with model:
    map_soln = xo.optimize()

To see how we did, let's plot the best fit solution. We don't have values for the coefficients (because we marginalized over them), so what we do to get a light curve is we *solve* for the coefficients (using `sys.solve()`), conditioned on the best fit values for the orbital parameters. We then set the map coefficients and compute the light curve model using `sys.flux()`. Note that we need to wrap some things in `xo.eval_in_model()` in order to get numerical values out of the `pymc3` model.

In [ ]:
with model:
    x = xo.eval_in_model(sys.solve(t=t)[0], point=map_soln)
    pri.map.amp = x[0]
    pri.map[1:, :] = x[1 : pri.map.Ny] / pri.map.amp
    sec.map.amp = x[pri.map.Ny]
    sec.map[1:, :] = x[pri.map.Ny + 1 :] / sec.map.amp
    flux_model = xo.eval_in_model(sys.flux(t=t), point=map_soln)

In [ ]:
model.d2logp()